
## Group-Aware alpha-Fair Allocation: closed form, gradients, tests


In [6]:
"""
============================================================
VERIFY group-wise α-fair closed form against CVXPY
(one global budget, any #groups, continuous doses)
============================================================
pip install torch cvxpy numpy
"""

import numpy as np
import cvxpy as cp
import torch


# --------------------------------------------------------------------
# CLOSED-FORM ALLOCATION   (covers α = 0,1,∞ + finite α)
# --------------------------------------------------------------------
def closed_form_group_alpha(
    b_hat: np.ndarray,
    cost: np.ndarray,
    group: np.ndarray,
    Q: float,
    alpha,
):
    """
    Parameters
    ----------
    b_hat : (N,T)  positive utilities
    cost  : (N,T)  positive costs
    group : (N,)   integers 0..K-1
    Q     : float  budget
    alpha : float >0 or 0 or 1 or np.inf
    Returns
    -------
    d_star : (N,T) optimal doses
    """
    N, T = b_hat.shape
    K = group.max() + 1

    # per-group best benefit-cost ratio ρ_k and its argmax (i*,t*)
    rho_k = np.zeros(K)
    idx_k = np.zeros((K, 2), dtype=int)
    for k in range(K):
        mask = group == k
        ratio = (b_hat[mask] / cost[mask]).reshape(-1)
        i_star = ratio.argmax()
        i_glob = np.where(mask)[0][i_star // T]
        t_glob = i_star % T
        rho_k[k] = ratio.max()
        idx_k[k] = [i_glob, t_glob]

    p_k = rho_k / np.bincount(group)  # p_k = ρ_k / G_k

    # ------------ Stage I: allocate budget B_k = x_k ------------
    if alpha == 0:  # utilitarian
        winners = np.flatnonzero(p_k == p_k.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:  # logarithmic
        x = np.full(K, Q / K)
    elif alpha == np.inf:  # max-min
        inv = 1 / p_k
        x = Q * inv / inv.sum()
    else:  # generic  (0<α<∞, α≠1)
        weights = p_k ** (1 / alpha - 1)
        x = Q * weights / weights.sum()

    # ------------ Stage II: spend each x_k on its best item -----
    d_star = np.zeros_like(b_hat)
    for k in range(K):
        i, t = idx_k[k]
        d_star[i, t] = x[k] / cost[i, t]

    return d_star


# --------------------------------------------------------------------
# CVXPY benchmark (same problem)
# --------------------------------------------------------------------
def solve_cvxpy(b_hat, cost, group, Q, alpha):
    N, T = b_hat.shape
    d = cp.Variable((N, T), nonneg=True)

    util = []
    for k in range(group.max() + 1):
        idx = group == k
        util_k = cp.sum(cp.multiply(b_hat[idx], d[idx])) / idx.sum()
        util.append(util_k)
    util = cp.hstack(util)

    if alpha == 0:
        obj = cp.Maximize(cp.sum(util))
    elif alpha == 1:
        obj = cp.Maximize(cp.sum(cp.log(util)))
    elif alpha == np.inf:
        obj = cp.Maximize(cp.min(util))
    else:
        obj = cp.Maximize(cp.sum(util ** (1 - alpha) / (1 - alpha)))

    prob = cp.Problem(obj, [cp.sum(cp.multiply(cost, d)) <= Q])
    prob.solve()
    return d.value


# --------------------------------------------------------------------
# quick numerical experiment
# --------------------------------------------------------------------
def one_run(N=4, T=2, K=3, Q=200.0, alpha=2.0, seed=0):
    rng = np.random.default_rng(seed)
    b = 1 + 4 * rng.random((N, T))
    c = 0.2 + 0.8 * rng.random((N, T))
    g = rng.integers(0, K, size=N)

    d_cf = closed_form_group_alpha(b, c, g, Q, alpha)
    d_cvx = solve_cvxpy(b, c, g, Q, alpha)
    gap = np.abs(d_cf - d_cvx).max()
    print('d_cf:\n', d_cf)
    print('d_cvx:\n', d_cvx)

    print(f"α={alpha:5},  max|closed-form − solver| = {gap:.3e}")


if __name__ == "__main__":
    for a in [0, 1, 0.5, 2.0, 5.0, np.inf]:
        one_run(alpha=a)


d_cf:
 [[  0.           0.        ]
 [  0.           0.        ]
 [  0.           0.        ]
 [  0.         587.32925844]]
d_cvx:
 [[  0.           0.        ]
 [  0.           0.        ]
 [  0.           0.        ]
 [  0.         587.32925844]]
α=    0,  max|closed-form − solver| = 0.000e+00
d_cf:
 [[  0.           0.        ]
 [  0.         329.72156311]
 [  0.         293.85603708]
 [  0.         195.77641948]]
d_cvx:
 [[3.81745185e-06 2.16776120e-06]
 [2.57304675e-06 3.29795889e+02]
 [2.48561565e-06 2.93822877e+02]
 [3.64641703e-06 1.95754339e+02]]
α=    1,  max|closed-form − solver| = 7.433e-02
d_cf:
 [[  0.           0.        ]
 [  0.         192.96529401]
 [  0.         334.32582847]
 [  0.         250.01492627]]
d_cvx:
 [[1.57646739e-06 7.20753469e-07]
 [1.21320711e-06 1.92956417e+02]
 [9.42114346e-07 3.34322609e+02]
 [1.62385165e-06 2.50022325e+02]]
α=  0.5,  max|closed-form − solver| = 8.877e-03
d_cf:
 [[  0.           0.        ]
 [  0.         413.15498793]
 [  0.      

In [8]:
import numpy as np

# -------------------------------------------------------------------------
#  SAFE CLOSED-FORM ALLOCATOR  (handles 1-column inputs without complaints)
# -------------------------------------------------------------------------
def closed_form_group_alpha(b_hat, cost, group, Q, alpha):
    """
    b_hat : (N,)  or (N,1) or (N,T)    strictly positive
    cost  : same shape as b_hat (broadcast OK)
    group : (N,)  or (N,1)  integer labels 0 … K-1
    Q     : scalar > 0
    alpha : 0, 1, np.inf, or positive float
    """
    # ------------ 1. normalise shapes ---------------------------------
    b_hat = np.asarray(b_hat, dtype=float)
    cost  = np.asarray(cost,  dtype=float)

    if b_hat.ndim == 1:                        # promote to 2-D (N,1)
        b_hat = b_hat[:, None]
    if cost.ndim == 1:
        cost  = cost[:, None]
    if cost.shape[1] == 1 and b_hat.shape[1] > 1:
        cost = np.repeat(cost, b_hat.shape[1], axis=1)
    if b_hat.shape[1] == 1 and cost.shape[1] > 1:
        b_hat = np.repeat(b_hat, cost.shape[1], axis=1)
    assert b_hat.shape == cost.shape, "benefit & cost must broadcast"

    # ------------ 2. squeeze group to 1-D int array -------------------
    group = np.asarray(group).astype(int).reshape(-1)
    if group.ndim != 1:
        raise ValueError("`group` must be 1-D after reshape")
    N, T = b_hat.shape
    if group.size != N:
        raise ValueError("length of `group` must equal #rows of b_hat")

    K  = group.max() + 1
    G  = np.bincount(group, minlength=K)       # each G_k > 0 ?

    # ------------ 3. best ratio per group ----------------------------
    rho   = np.empty(K)
    idx_k = np.empty((K, 2), dtype=int)

    for k in range(K):
        rows = np.flatnonzero(group == k)
        ratio_sub = b_hat[rows] / cost[rows]   # shape (|rows|, T)
        flat_idx  = ratio_sub.argmax()         # 0 … |rows|·T−1
        r_loc, t_star = divmod(flat_idx, T)
        i_star = rows[r_loc]
        rho[k]  = ratio_sub.flat[flat_idx]
        idx_k[k] = (i_star, t_star)

    p = rho / G                                # p_k = ρ_k / G_k

    # ------------ 4. allocate budgets x_k ----------------------------
    if alpha == 0:                             # utilitarian
        winners = np.flatnonzero(p == p.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:                           # log utility
        x = np.full(K, Q / K)
    elif alpha == np.inf:                      # max–min
        inv = 1 / p
        x = Q * inv / inv.sum()
    else:                                      # generic α
        beta   = 1.0 / alpha
        w      = p ** (beta - 1)
        x = Q * w / w.sum()

    # ------------ 5. build decision matrix ---------------------------
    d_star = np.zeros_like(b_hat)
    for k, (i, t) in enumerate(idx_k):
        d_star[i, t] = x[k] / cost[i, t]

    return d_star, idx_k, x, rho


# -------------------------------------------------------------------------
# quick demo  : 5000×1  arrays
# -------------------------------------------------------------------------
if __name__ == "__main__":
    N, T, K, Q = 5000, 1, 2, 1000.0
    rng = np.random.default_rng(42)
    benefit = 1 + rng.random((N, T))
    cost    = 0.2 + 0.8 * rng.random((N, T))
    race    = rng.integers(0, K, size=(N, 1))   # note shape (N,1) !

    d_opt, idx, x, rho = closed_form_group_alpha(
        benefit, cost, race, Q, alpha=2.0
    )
    print("budgets per group:", x.round(2))
    print("decision matrix shape:", d_opt.shape, " ; non-zeros:",
          np.count_nonzero(d_opt))


budgets per group: [497.94 502.06]
decision matrix shape: (5000, 1)  ; non-zeros: 2


In [7]:
"""
group_alpha_derivs.py
=====================

Closed form, objective gradient, and Jacobian for
  • single global budget Q
  • continuous decision
  • any alpha in {0, 1, ∞} U (0, ∞)
  • arbitrary number of groups K (ties assumed measure-zero)

Run this file to see a finite-difference sanity check.
"""

import numpy as np


# ---------------------------------------------------------------------
# closed-form allocation  (two-stage: pick best item per group, then budget)
# ---------------------------------------------------------------------
def closed_form_group_alpha(b, c, g, Q, alpha):
    """
    Parameters
    ----------
    b : (N,T) positive predicted benefit   (numpy)
    c : (N,T) positive cost
    g : (N,)  integer group labels 0..K-1
    Q : float budget
    alpha : float >0   or 0   or 1   or np.inf
    Returns
    -------
    d_star : (N,T) allocation (numpy)
    idx_k  : (K,2) index (i,t) of best item for each group (needed later)
    """
    N, T = b.shape
    K = g.max() + 1

    # ----- best item (i_k,t_k) & ratio ρ_k in each group --------------
    rho = np.zeros(K)
    idx_k = np.zeros((K, 2), dtype=int)

    for k in range(K):
        mask = g == k
        ratio = (b[mask] / c[mask]).reshape(-1)
        flat_idx = ratio.argmax()
        i_global = np.where(mask)[0][flat_idx // T]
        t_global = flat_idx % T
        idx_k[k] = i_global, t_global
        rho[k] = ratio.max()

    G = np.bincount(g, minlength=K)
    p = rho / G                       # p_k = ρ_k / G_k

    # ---------- Stage I : budgets x_k = B_k ---------------------------
    if alpha == 0:                    # utilitarian
        winners = np.flatnonzero(p == p.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:                  # log utility
        x = np.full(K, Q / K)
    elif alpha == np.inf:             # max–min
        inv = 1 / p
        x = Q * inv / inv.sum()
    else:                             # generic α
        beta = 1.0 / alpha
        weights = p ** (beta - 1)     # p^{1/α - 1}
        x = Q * weights / weights.sum()

    # ---------- Stage II : spend within group on best (i,t) -----------
    d_star = np.zeros_like(b)
    for k, (i, t) in enumerate(idx_k):
        d_star[i, t] = x[k] / c[i, t]

    return d_star, idx_k, p, x


# ---------------------------------------------------------------------
# gradient of objective  W(b)  w.r.t. each b_{it}
# ---------------------------------------------------------------------
def grad_W_wrt_b(b, c, g, Q, alpha):
    """
    Returns  ∇_b W  with the same shape as b
    """
    d_star, idx_k, p, _ = closed_form_group_alpha(b, c, g, Q, alpha)
    K = len(p)
    G = np.bincount(g, minlength=K)
    grad = np.zeros_like(b)

    if alpha in (0, np.inf):
        # objective is non-smooth here; return NaNs
        grad[:] = np.nan
        return grad

    if alpha == 1:                                  # W = Σ log u_k
        for k, (i, t) in enumerate(idx_k):
            grad[i, t] = 1 / (p[k] * G[k] * c[i, t])
        return grad

    # ---------- generic  0<α<∞, α≠1  -------------------------------
    beta = 1.0 / alpha
    D = np.sum(p ** (beta - 1))                     # denominator
    u = Q * p ** beta / D                          # group utilities

    # ∂W/∂u_k  and helper coeffs
    dW_du = u ** (-alpha)                          # u_k^{−α}
    coeff = Q ** (-alpha) * D ** (alpha - 2)       # common front factor

    for k, (i, t) in enumerate(idx_k):
        pk = p[k]
        term = (beta * D - (beta - 1) * pk ** (beta - 1))
        dW_dpk = coeff * pk ** (beta - 2) * term
        grad[i, t] = dW_dpk / (G[k] * c[i, t])     # dp/db = 1/(G_k c)
    return grad


# ---------------------------------------------------------------------
# full Jacobian  ∂d*/∂b   (sparse dictionary representation)
# ---------------------------------------------------------------------
def jacobian_d_wrt_b(b, c, g, Q, alpha):
    """
    Returns
    -------
    J : dict mapping (i*,t*)  ->  gradient row (N,T) as numpy array
        Only K rows are non-zero: one per group winner (i*,t*).
    """
    d_star, idx_k, p, x = closed_form_group_alpha(b, c, g, Q, alpha)
    K = len(p)
    G = np.bincount(g, minlength=K)
    N, T = b.shape
    J = {}

    # special / non-smooth cases --------------------------------------
    if alpha in (0, 1, np.inf):
        # Jacobian exists but is piecewise-constant & sparse:
        #  ∂d*(winner k)/∂b(winner k) via budget split; others zero.
        # Users typically rely on sub-gradients → return NaNs.
        for k, (i, t) in enumerate(idx_k):
            J[(i, t)] = np.full_like(b, np.nan)
        return J

    # ---------- generic  0<α<∞, α≠1 ----------------------------------
    beta = 1.0 / alpha
    D = np.sum(p ** (beta - 1))
    dD_dpk = (beta - 1) * p ** (beta - 2)          # derivative of D
    # pre-compute    ∂x_l / ∂p_k   for every pair (l,k)
    x_grad = np.zeros((K, K))
    for l in range(K):
        for k in range(K):
            if k == l:
                num = (beta - 1) * p[k] ** (beta - 2) * D
                num -= p[k] ** (beta - 1) * dD_dpk[k]
                x_grad[l, k] = Q * num / D ** 2
            else:
                x_grad[l, k] = -Q * p[l] ** (beta - 1) * dD_dpk[k] / D ** 2

    # build Jacobian rows (only one non-zero col per group)
    for k, (i_win, t_win) in enumerate(idx_k):
        row = np.zeros_like(b)
        # effect of p_k on every x_l  (thus on every winner l)
        for l, (i_l, t_l) in enumerate(idx_k):
            row[i_l, t_l] += (
                x_grad[l, k] / c[i_l, t_l] / (G[k] * c[i_win, t_win])
            )
        J[(i_win, t_win)] = row
    return J


# ---------------------------------------------------------------------
# quick finite-difference check  (generic α)
# ---------------------------------------------------------------------
def finite_difference_check():
    N, T, K = 10, 3, 3
    Q, alpha = 100.0, 2.0
    rng = np.random.default_rng(0)
    b = 1 + 4 * rng.random((N, T))
    c = 0.3 + 0.7 * rng.random((N, T))
    g = rng.integers(0, K, size=N)

    # gradient of W ---------------------------------------------------
    grad_analytic = grad_W_wrt_b(b, c, g, Q, alpha)
    eps = 1e-6
    grad_fd = np.zeros_like(b)
    for i in range(N):
        for t in range(T):
            b_plus = b.copy()
            b_minus = b.copy()
            b_plus[i, t] += eps
            b_minus[i, t] -= eps
            W_plus = objective_value(b_plus, c, g, Q, alpha)
            W_minus = objective_value(b_minus, c, g, Q, alpha)
            grad_fd[i, t] = (W_plus - W_minus) / (2 * eps)
    print("max|∇W_fd − ∇W_an| =", np.abs(grad_fd - grad_analytic).max())

    # Jacobian of d* --------------------------------------------------
    J = jacobian_d_wrt_b(b, c, g, Q, alpha)
    worst = 0.0
    for (i0, t0), row in J.items():
        b_eps = b.copy()
        b_eps[i0, t0] += eps
        d_plus, *_ = closed_form_group_alpha(b_eps, c, g, Q, alpha)
        d_orig, *_ = closed_form_group_alpha(b, c, g, Q, alpha)
        fd_col = (d_plus - d_orig) / eps
        worst = max(worst, np.abs(fd_col - row).max())
    print("max|Jac_fd − Jac_an| =", worst)


def objective_value(b, c, g, Q, alpha):
    d, *_ = closed_form_group_alpha(b, c, g, Q, alpha)
    K = g.max() + 1
    G = np.bincount(g, minlength=K)
    util = np.zeros(K)
    for k in range(K):
        i, t = np.where((g == k).reshape(-1, 1))
    # faster: read utilities from d:
    for k in range(K):
        mask = g == k
        util[k] = (b[mask] * d[mask]).sum() / G[k]

    if alpha == 0:
        return util.sum()
    if alpha == 1:
        return np.log(util).sum()
    if alpha == np.inf:
        return util.min()
    return np.sum(util ** (1 - alpha) / (1 - alpha))


# ---------------------------------------------------------------------
if __name__ == "__main__":
    finite_difference_check()


max|∇W_fd − ∇W_an| = 0.0029929353122838793
max|Jac_fd − Jac_an| = 1.4945091919571496e-06
